##Pyruvate Oxidase (PoxB) Tranformation Code.
Python workflow to generate the crystal structure PDB ID: 3EY9 from its asymmetrical unit using symmetry operations.


**Download the PDB File from RCSB**

Fetch the PDB file directly from the RCSB database using its PDB ID

In [ ]:
pdb_id = "3EY9"
url = f"https://files.rcsb.org/download/{pdb_id}.pdb"

pdb_text = requests.get(url).text
with open("3EY9.pdb", "w") as f:
    f.write(pdb_text)

print("PDB downloaded.")


**Read Atoms from Asymmetric Unit**

The ASU is the smallest unique set of coordinates stored in the PDB file.
We extract only ATOM and HETATM records and store:
1. The original PDB line (for formatting)
2. The Cartesian coordinates (x, y, z) as a NumPy vector

In [ ]:
atoms = []

with open("3EY9.pdb") as f:
    for line in f:
        if line.startswith(("ATOM", "HETATM")):
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])
            atoms.append((line, np.array([x, y, z])))

print(f"Read {len(atoms)} atoms from asymmetric unit.")

**Apply Symmetry Operators (REMARK 290, P 43 21 2) and Write Files**  


In [ ]:
sym_ops = [
    # 1:  X,  Y,  Z
    (np.array([[ 1,  0,  0],
               [ 0,  1,  0],
               [ 0,  0,  1]]),
     np.array([0.000, 0.000, 0.000])),

    # 2: -X, -Y,  Z + 1/2
    (np.array([[-1,  0,  0],
               [ 0, -1,  0],
               [ 0,  0,  1]]),
     np.array([0.000, 0.000, 76.870])),

    # 3: -Y + 1/2,  X + 1/2,  Z + 3/4
    (np.array([[ 0, -1,  0],
               [ 1,  0,  0],
               [ 0,  0,  1]]),
     np.array([75.685, 75.685, 115.305])),

    # 4:  Y + 1/2, -X + 1/2,  Z + 1/4
    (np.array([[ 0,  1,  0],
               [-1,  0,  0],
               [ 0,  0,  1]]),
     np.array([75.685, 75.685, 38.435])),

    # 5: -X + 1/2,  Y + 1/2, -Z + 3/4
    (np.array([[-1,  0,  0],
               [ 0,  1,  0],
               [ 0,  0, -1]]),
     np.array([75.685, 75.685, 115.305])),

    # 6:  X + 1/2, -Y + 1/2, -Z + 1/4
    (np.array([[ 1,  0,  0],
               [ 0, -1,  0],
               [ 0,  0, -1]]),
     np.array([75.685, 75.685, 38.435])),

    # 7:  Y,  X, -Z
    (np.array([[ 0,  1,  0],
               [ 1,  0,  0],
               [ 0,  0, -1]]),
     np.array([0.000, 0.000, 0.000])),

    # 8: -Y, -X, -Z + 1/2
    (np.array([[ 0, -1,  0],
               [-1,  0,  0],
               [ 0,  0, -1]]),
     np.array([0.000, 0.000, 76.870]))
]


# Apply symmetry & write files

for i, (R, t) in enumerate(sym_ops, start=1):
    out_lines = []

    for line, coord in atoms:
        new_coord = R @ coord + t
        new_line = (
            line[:30] +
            f"{new_coord[0]:8.3f}{new_coord[1]:8.3f}{new_coord[2]:8.3f}" +
            line[54:]
        )
        out_lines.append(new_line)

    filename = f"3EY9_symop_{i}.pdb"
    with open(filename, "w") as f:
        f.writelines(out_lines)

    print(f"Wrote {filename}")